# COVID-19 Data Analysis and Visualization



# What is this Notebook about?

In this Jupyter notebook,I have show the step-by-step code of creating impactful visualizations and charts using  Python programming.

# Install libraries

In [ ]:
!pip install dash
!pip install jupyter-dash


# Import libraries

In [ ]:
from dash_html_components.H2 import H2
from dash_html_components.P import P
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [ ]:
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()
app.config.suppress_callback_exceptions = True

link='https://api.covid19india.org/csv/latest/state_wise_daily.csv'
total_state_wise_daily =  pd.read_csv(link)


header= ['date','Date','Status','Total','Andaman and Nicobar Islands','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar',
'Chandigarh','Chhattisgarh','Dadra and Nagar Haveli','Daman and Diu','Delhi','Goa','Gujarat','Haryana','Himachal Pradesh',
'Jammu and Kashmir','Jharkhand','Karnataka','Kerala','Ladakh','Lakshadweep','Madhya Pradesh','Maharashtra','Manipur',
'Meghalaya','Mizoram','Nagaland','Odisha','Puducherry','Punjab','Rajasthan','Sikkim','Tamil Nadu','Telangana','Tripura ',
 'Uttar Pradesh','Uttarakhand','West Bengal','Unknown']

total_state_wise_daily.columns=header

state_list=[ 'Andaman and Nicobar Islands', 'Andhra Pradesh',  'Arunachal Pradesh','Assam','Bihar','Chandigarh','Chhattisgarh','Dadra and Nagar Haveli',
 'Daman and Diu','Delhi','Goa','Gujarat','Haryana','Himachal Pradesh','Jammu and Kashmir','Jharkhand','Karnataka','Kerala','Ladakh',
 'Lakshadweep','Madhya Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland','Odisha','Puducherry','Punjab','Rajasthan',
'Sikkim','Tamil Nadu','Telangana','Tripura ','Uttar Pradesh','Uttarakhand','West Bengal']

type_list=['Confirmed','Recovered','Deceased']


def df(state):
     
    coun= total_state_wise_daily[['Date','Status','Total']] 
     
    stat=total_state_wise_daily[['Date','Status',state]]
      
    return coun,stat 


# Application layout
app.layout = html.Div(children=[ 
                                 html.H1('Covid-19 Data Analysis and Visualization', 
                                 style={'textAlign': 'center', 'color': '#503D36','font-size': 24}),
                                 
                                 html.H2('Daily covid cases'),
                                 html.P('Please select the state you want to view data of:- '),
                
                                html.Div([
                                    html.Div([ html.Div([ html.H2('State:', style={'margin-right': '2em'})]),
                                                           dcc.Dropdown(id='state-type', 
                                                                        options=[{'label': i, 'value': i} for i in state_list],
                                                                        placeholder="Select state ",
                                                                        style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}), 
                                             ], style={'display':'flex'})
                                    
                                         ]),

                                html.Div([
                                        html.Div([ ], id='coun'),
                                         ]),

                                html.Div([
                                        html.Div([ ], id='state'),
                                        ]),

                                ])


                                  
                                
                                

@app.callback(
             [ 
                Output(component_id='coun', component_property='children'),
                Output(component_id='state', component_property='children'),
              ], 
              [Input(component_id='state-type', component_property='value'),
              ]
              
              

             )

def get_graph(state):

    co,st=df(state)    

    coun_fig = px.line(co, x='Date',color='Status', y='Total', title='Total Confirmed,recovery,Deaths cases in India')
    state_fig = px.line(st, x='Date', y=state,color='Status', title='Total Confirmed,recovery,Deaths in State')
 
                   
    return [
            dcc.Graph(figure=coun_fig), 
            dcc.Graph(figure=state_fig),        
           ]


# Run the app
if __name__ == '__main__': 
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2021 13:00:03] "GET /_alive_5ec9a005-d9bb-43ed-b2fc-865922d31324 HTTP/1.1" 200 -


<IPython.core.display.Javascript object>

# My COVID-19 Github repo
Other COVID-19 related info is in [ my Github repo here](https://github.com/divyansh1110/COVID-19-Data-Analysis-and-Visualization)
